## Vova Bravo

### Hutchinson estimator of the trace

In [1]:
import numpy as np

In [2]:
def t_comp(eps, delta):
    fig = 8./(eps**2) * np.log(2./delta)
    fig = int(fig+1)
    return fig

In [3]:
def c(p):
    fig = (2 + int(p))**(p+1)
    if p > 1:
        nafig = np.prod(np.array([abs((p - i + 1)/i) for i in range(1, int(p) + 1)]))
    else:
        nafig = 1
    return fig*nafig

In [4]:
def m_comp(c, p, eps, n):
    fig = 7* np.power(3*c*n/(p*eps), 1/p)
    return np.int(fig + 1)

In [5]:
def beta(m, c, p):
    fig = (c/p) * np.power(float(m)+1, -p)
    return fig

In [6]:
def hutch(X, p, m, t):
    y = []
    n = X.shape[0]
    for i in range(1,t+1):
        g_i = np.random.binomial(1, 0.5, size=(n,)) * 2 - 1
        v_k = X@g_i
        u_k = g_i@v_k
        a_k = p
        S_i_k = a_k*u_k
        for k in range(2, m+1):
            v_k = X@v_k
            u_k = g_i@v_k
            a_k = a_k* (p-(k-1))/k
            S_i_k = S_i_k + (((-1)**(k-1)) * a_k) * u_k
        y.append(S_i_k)
    y = np.array(y).mean()
    return y

In [7]:
def power_method(A, x0, num_iter): # 5 pts
    # enter your code here
    x = np.copy(x0)
    A_x = np.squeeze(np.array(A.dot(x)))
    l = x.T @ A_x
    for i in range(num_iter):
        x = np.squeeze(np.array(A.dot(x)))
        x = x / np.linalg.norm(x)
        A_x = np.squeeze(np.array(A.dot(x)))
        l = x.T @ A_x
    return x, l

In [8]:
def alpha(A, delta):
    n = A.shape[0]
    q = int(4.82 * np.log(1. / delta) + 1)
    t = int(0.5 * np.log(4 * n) + 1)
    max_lambda = 0
    for _ in range(q):
        x0 = np.random.binomial(1, 0.5, size=(n,)) * 2 - 1
        x, l = power_method(A, x0, t)
        if l > max_lambda:
            max_lambda = l
    return max_lambda

In [9]:
def vova_bravo_without(A, p, eps, delta):
    n = A.shape[0]
    t = t_comp(eps, delta)
    c_p = c(p)
    m = m_comp(c_p, p, eps, n)
    b_m = beta(m, c_p, p)
    a = alpha(A, delta)
    return np.power(a, p) * int((1+b_m)*n - hutch(np.eye(n) - A / a, p, m, t))

In [10]:
def compute_true_schatten(A, p):
    u, s, vh = np.linalg.svd(A)
    return np.power(s, p).sum()

In [11]:
A = np.random.normal(size=(200, 200))
A = A.T @ A

In [12]:
p = 5
schatten_vova_bravo = vova_bravo_without(A, p, eps=0.05, delta=0.01)
schatten_true = compute_true_schatten(A, p)

In [13]:
np.abs((schatten_vova_bravo - schatten_true) / schatten_true)

0.05434399077159538

In [14]:
import scipy.special

In [15]:
def h_tilde(A, m, p):
    A_k = A
    h_tilde = np.zeros_like(A)
    for i in range(1, m+1):
        h_tilde += (-1 if i % 2 == 0 else 1) * scipy.special.binom(p, i) * A_k
        A_k = A_k @ A
    return h_tilde

In [16]:
p = 5
eps = 0.05
delta = 0.01
n = A.shape[0]
t = t_comp(eps, delta)
c_p = c(p)
m = m_comp(c_p, p, eps, n)
b_m = beta(m, c_p, p)
a = 6 * alpha(A, delta)

In [ ]:
hutch(np.eye(n) - A / a, p, m, t)

In [ ]:
def hutch_test(A, eps, delta):
    p = int(20*np.log(2/delta)/(eps)**2)
    gamma = 0
    for i in range(p):
        g = np.random.randn(A.shape[0])
        fig = A.dot(g)
        nafig = g.dot(fig)
        gamma = gamma + nafig
    gamma = gamma/p
    return gamma

In [ ]:
h_tilde(np.eye(n) - A / a, m, p).trace()

In [ ]:
hutch_test(h_tilde(np.eye(n) - A / a, m, p), eps, delta)